### LAB 1 : Big Data In Media Technology 
#### Sentiment Classification with Naïve Bayesian Classifier

This lab aims to understand naïve bayesian classification technique, get familiar with handly such a type of data, do data cleaning and data analysis, learn to employ python to do analysis.  

### Plan :
    1. Data Exploration 
    2. Data Processing using NLTK 
    3. Bag of Words & TF-IDF models 
    4. Train and test the model 

### 1. Data Exploration 

#### 1.1 import python useful packages 

In [1]:
!pip install nltk

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
import numpy as np
import pandas as pd 
import sklearn as skl
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings


plt.rcParams['figure.figsize'] = (12,12)
if not sys.warnoptions:
    warnings.simplefilter("ignore")

#### 1.2 import data from csv to dataframe

In [3]:
''' Function _ csv_pd : @input : String = filename => CSV file name
                        @ouput : dataFrame 
'''              
def csv_pd(filename) :
    out = pd.read_csv(filename,engine="python",sep=",")
    return out 

In [4]:
train_data = csv_pd("train.csv")
train_data.head(5)

,score,text
0,0,"overgeneralized, not helpful to anyone serious..."
1,1,Great sound and service.
2,1,love this book!!!: this book is a fast read ab...
3,1,A hugely enjoyable screen version of Rona Jaff...
4,0,What an uninteresting hodge-podge. It could ha...


In [5]:
## Verify that scores can be 0 or 1, Binary Classification
train_data.score.unique()

array([0, 1])

### 2.Processing Data using NLTK

In [43]:
# Handling Missing Values : 
def fillna(df) :
    show = df.count() - df.isna().count()
    print("Here's the count list of missing values {}".format(show.to_frame()))
    values = {"score": 0, "text": "Negative"}
    df.fillna(value=values)

In [44]:
fillna(train_data)

Here's the count list of missing values        0
score  0
text   0


In [6]:
# Remove puntuations from text 
import string
import re

def remove_puncts(text) :
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub("\d+", "", text)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = re.sub(emoji_pattern, '', text)
    return text 

def tokenize(text) :
    text = text.split()
    return text

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
def remove_stop_words(tokenize_text) :
    tokenize_text = [w.lower() for w in tokenize_text if not w.lower() in stop_words]
    return tokenize_text

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

def lemmating(tokenize_text):
    tokenize_text = [wn.lemmatize(word) for word in tokenize_text]
    return tokenize_text

ps = nltk.PorterStemmer()
ls = nltk.LancasterStemmer()

def stemming(tokenize_text):
    tokenize_text = [ps.stem(word) for word in tokenize_text]
    tokenize_text = [ls.stem(word) for word in tokenize_text]
    return tokenize_text

[nltk_data] Downloading package stopwords to
[nltk_data]     /mnt/workspace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /mnt/workspace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def apply_processing(df):
    df["without puncts"] =df["text"].apply(lambda x: remove_puncts(x))
    df["tokenize text"] = df["without puncts"].apply(lambda x: tokenize(x))
    df["without stop words"] = df["tokenize text"].apply(lambda x: remove_stop_words(x))
    df["lemmatized"] = df["without stop words"].apply(lambda x: lemmating(x))
    df["stemming"] = df["lemmatized"].apply(lambda x:stemming(x))
    df["processed"] = df["stemming"].apply(lambda x: ' '.join(str(e) for e in x))
    return df
    
train_processed = apply_processing(train_data)

In [8]:
train_processed.head(5)

,score,text,without puncts,tokenize text,without stop words,lemmatized,stemming,processed
0,0,"overgeneralized, not helpful to anyone serious...",overgeneralized not helpful to anyone seriousl...,"[overgeneralized, not, helpful, to, anyone, se...","[overgeneralized, helpful, anyone, seriously, ...","[overgeneralized, helpful, anyone, seriously, ...","[overg, help, anyon, sery, appl, prem, know, n...",overg help anyon sery appl prem know noth mean...
1,1,Great sound and service.,Great sound and service,"[Great, sound, and, service]","[great, sound, service]","[great, sound, service]","[gre, sound, serv]",gre sound serv
2,1,love this book!!!: this book is a fast read ab...,love this book this book is a fast read about ...,"[love, this, book, this, book, is, a, fast, re...","[love, book, book, fast, read, poor, young, fa...","[love, book, book, fast, read, poor, young, fa...","[lov, book, book, fast, read, poor, young, far...",lov book book fast read poor young farm boy gr...
3,1,A hugely enjoyable screen version of Rona Jaff...,A hugely enjoyable screen version of Rona Jaff...,"[A, hugely, enjoyable, screen, version, of, Ro...","[hugely, enjoyable, screen, version, rona, jaf...","[hugely, enjoyable, screen, version, rona, jaf...","[hug, enjoy, screen, vert, ron, jaff, bestsel,...",hug enjoy screen vert ron jaff bestsel potboil...
4,0,What an uninteresting hodge-podge. It could ha...,What an uninteresting hodgepodge It could have...,"[What, an, uninteresting, hodgepodge, It, coul...","[uninteresting, hodgepodge, could, something, ...","[uninteresting, hodgepodge, could, something, ...","[uninterest, hodgepodg, could, some, imagin, s...",uninterest hodgepodg could some imagin seem go...


### 3. Bag_of_Words and TF-IFD 

#### 3.1 Bag of Words Model 
A bag-of-words is a representation of text that describes the occurrence of words within a document. It involves two things:
    - A vocabulary of known words.
    
    - A measure of the presence of known words.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

countV = CountVectorizer() # Bag Of Words
train_countV = countV.fit_transform(train_processed.processed) # Fit the dictionnary

In [13]:
print(countV)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


#### 3.2 TF-IDF Model 

In [134]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfV = TfidfVectorizer()
train_tfidf = tfidfV.fit_transform(train_processed.processed)

### 4. Train and Test The models 

In [12]:
## Import Datas Test and Evaluation
test_data = csv_pd("test.csv")
evaluation_data = csv_pd("evaluation.csv")

## Process Datas Test and Evaluation 
test_processed = apply_processing(test_data)
evaluation_processed = apply_processing(evaluation_data)

#### 4.1 Choose NLP Model (BoW or TF-IDF) wrt Naives Bayes Classifier

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

def test_model_BoW(train_df,test_df,y,y_test) : 
    model_bow = Pipeline([('countV_bayes',countV),('bayes_classifier',MultinomialNB())])
    model_bow.fit(train_df,y)
    y_pred_train = model_bow.predict(train_df)
    y_pred_test = model_bow.predict(test_df)
    accuracy_train = np.mean(y_pred_train == y)
    accuracy_test = np.mean(y_pred_test == y_test)
    print("For training score Using BoW We reached {} as accuracy".format(accuracy_train))
    print("For testing score Using BoW We reached {} as accuracy".format(accuracy_test))
    print("#################End BoW with Naive Bayes#################")
    
    return y_pred_train,y_pred_test
    
def test_model_TFIDF(train_df,test_df,y,y_test) : 
    model_TFIDF = Pipeline([('tfidfv_bayes',tfidfV),('bayes_classifier',MultinomialNB())])
    model_TFIDF.fit(train_df,y)
    y_pred_train = model_TFIDF.predict(train_df)
    y_pred_test = model_TFIDF.predict(test_df)
    accuracy_train = np.mean(y_pred_train == y)
    accuracy_test = np.mean(y_pred_test == y_test)
    print("For training score Using BoW We reached {} as accuracy".format(accuracy_train))
    print("For testing score Using BoW We reached {} as accuracy".format(accuracy_test))
    print("#################End TF-IDF with Naive Bayes#################")
    
    return y_pred_train,y_pred_test


In [138]:
bowtrain, bowtest = test_model_BoW(train_processed.processed,test_processed.processed,train_processed.score,test_processed.score)

For training score Using BoW We reached 0.9186666666666666 as accuracy
For testing score Using BoW We reached 0.816 as accuracy
#################End BoW with Naive Bayes#################


In [140]:
tftrain, tftest = test_model_TFIDF(train_processed.processed,test_processed.processed,train_processed.score,test_processed.score)

For training score Using BoW We reached 0.9248 as accuracy
For testing score Using BoW We reached 0.8216 as accuracy
#################End TF-IDF with Naive Bayes#################


In [143]:
## Display Confusion Matrix 
from sklearn.metrics import confusion_matrix
confusion_matrix(train_data.score, bowtrain)

array([[3587,  161],
       [ 403, 3349]])

In [16]:
tftrain, tftest = test_model_BoW(train_processed.processed,evaluation_processed.processed,train_processed.score,evaluation_processed.score)

For training score Using BoW We reached 0.9186666666666666 as accuracy
For testing score Using BoW We reached 0.7952 as accuracy
#################End BoW with Naive Bayes#################
